# requests cookie（验证码）登陆古诗文网

古诗文网需要通过登陆，进入到主页面。

登陆有验证码

难点：

1. 隐藏域
2. 验证码

In [ ]:
# 通过登陆，进入到主页面

# 登陆有验证码

# 获取登陆接口的方式：
# 1. 输入一个错误的用户名和密码（登陆失败不会跳转到新的界面），检查页面源码，找到登陆接口
# 2. 通过F12->Network->(旧版本chrome需要勾选Preserve log)->进行正常登陆操作->登陆成功后查看登陆接口

# 登陆接口一般名字包含login  且payload中有登陆信息
# https://www.gushiwen.cn/user/login.aspx?from=http%3a%2f%2fwww.gushiwen.cn%2fuser%2fcollect.aspx


# 通过找登陆接口我们发现 登陆的时候需要的参数很多：
# __VIEWSTATE: fbw1myRx1Mn0oRTJrYTU8KxJ8QK+luQjc1Z5O9Zh8BM++V+NhKPibtPrUeNdM2vo3KBJLWdprzFwimOQ4cF072Pg7ibDZN3UaPmaejyoULJWILHk9czhE3MEsCii5HDmTnPNcTSBMn85XgzZT+8llLYd0wU=
# __VIEWSTATEGENERATOR: C93BE1AE
# from: http://www.gushiwen.cn/user/collect.aspx
# email: afasg@163.com
# pwd: asf32513215
# code: EX6q
# denglu: 登录

# 我们观察到_VIEWSTATE  __VIEWSTATEGENERATOR    code 是可以变化的量

# 难点：（1）_VIEWSTATE __VIEWSTATEGENERATOR    一般情况看不到的数据都是在页面的源码中
#           我们观察到这两个数据在页面的源码中 所以我们需要获取页面的源码 然后进行解析就可以获取了
#           这两个变量放在隐藏域中<input type='hidden'>，页面上没有显示，但是在源码中可以看到
#      （2）验证码
#          验证码是一个图片，我们需要下载这个图片，然后手动输入

In [10]:
import requests

# 这是登陆页面的url
url = 'https://www.gushiwen.cn/user/login.aspx?from=http%3a%2f%2fwww.gushiwen.cn%2fuser%2fcollect.aspx'

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'
}

# 获取页面的源码
response = requests.get(url=url, headers=headers)
content = response.text

print(content)


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" id="html">
<head><meta http-equiv="Cache-Control" content="no-siteapp" /><meta http-equiv="Cache-Control" content="no-transform " /><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><title>
登录古诗文网
</title>
<script type="text/javascript">if ((navigator.userAgent.match(/(phone|pad|pod|iPhone|iPod|ios|iPad|Android|Mobile|BlackBerry|IEMobile|MQQBrowser|JUC|Fennec|wOSBrowser|BrowserNG|WebOS|Symbian|Windows Phone)/i))) {window.location.href ="https://m.gushiwen.cn/app/DefaultGwd.aspx";} else {}
</script>
<link href="/user/reg.css" rel="stylesheet" type="text/css" />
<script src="/js/Code.js" type="text/javascript"></script>
<script src="/js/jquery-3.2.1.min.js" type="text/javascript"></script>
<style type="text/css">.hide-centerlogin{width:258px;height:250px;right:0;left:0;bottom:0;top:0;margin:auto;z-index:1;

In [11]:
# 解析页面源码，获取__VIEWSTATE __VIEWSTATEGENERATOR
# 用xpath或bs4解析页面源码

from bs4 import BeautifulSoup

soup = BeautifulSoup(content, 'lxml')

# 获取__VIEWSTATE
__VIEWSTATE = soup.select('#__VIEWSTATE')[0].attrs.get('value')

# 获取__VIEWSTATEGENERATOR
__VIEWSTATEGENERATOR = soup.select('#__VIEWSTATEGENERATOR')[0].attrs.get('value')

print(__VIEWSTATE)
print(__VIEWSTATEGENERATOR)


d7jya7pwrydVEqQTf+1k6bEs9wcgwrzhLYiOfJ6MGE4L51iXAQX/iheLkLq95d7zWi4gr+WkEbom8xPvXIZqYchiWtDop/6xmoLNNZ2ItkaGEh5B4Geoj0wtVQF9volUPrq+Tv1glFaKIxG3TAIMh92DEdE=
C93BE1AE


In [ ]:
# 获取验证码图片

# 通过‘检查’看到验证码是一个img
# <img id="imgCode" style="cursor: pointer; float:left; margin-left:5px; margin-top:1px;" width="60" height="27" src="/RandCode.ashx" onclick="GetCodeImg()" alt="看不清，换一张">

code = soup.select('#imgCode')[0].attrs.get('src')
print(code) # /RandCode.ashx

code_url = 'https://www.gushiwen.cn' + code
print(code_url)

# 下载验证码图片
# 有坑 当前验证码的获取和后续的访问不是一个session，所以不是一个验证码。
# import urllib.request
# urllib.request.urlretrieve(url=code_url, filename='code.jpg')

# requests里面有一个方法 session()  通过session的返回值，就能使请求变成一个对象。
session = requests.session()
# 验证码的url的内容
response_code = session.get(code_url)
# 注意此时不能用.text而要用.content i.e., 使用二进制数据，因为我们要使用的是图片的下载
content_code = response_code.content
# wb的模式就是将二进制数据写入文件
with open('code.jpg', 'wb') as fp:
    fp.write(content_code)


/RandCode.ashx
https://www.gushiwen.cn/RandCode.ashx


In [ ]:

# 获取验证码的图片之后，下载到本地，然后观察验证码，然后在控制台输入验证码 就可以将这个值给
# code的参数 就可以登陆

code_name = input('请输入你的验证码：')

# 点击登陆
url_post = 'https://www.gushiwen.cn/user/login.aspx?from=http%3a%2f%2fwww.gushiwen.cn%2fuser%2fcollect.aspx'

data_post = {
    '__VIEWSTATE': __VIEWSTATE,
    '__VIEWSTATEGENERATOR': __VIEWSTATEGENERATOR,
    'from': 'http://www.gushiwen.cn/user/collect.aspx',
    'email': '595165358@qq.com',
    'pwd': 'action',
    'code': code_name,
    'denglu': '登录'
}


response_post = session.post(url=url_post, headers=headers, data=data_post)

content_post = response_post.text

with open('gushiwen.html', 'w', encoding='utf-8') as fp:
    fp.write(content_post)


In [ ]:
# 总结
# 难点
# （1）隐藏域
# （2）验证码